# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-87f3f9000e2-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("International Treaties") 

Consider the following exploratory information need:

> You investigating existing international treaties

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P39`     | position held   | predicate |
| `wd:Q131569`  | treaty | node      |
| `wd:Q46`      | Europe        | node |
| `wd:Q38`      | Italy          | node |
| `wd:Q21707860` | Paris Agreement           | node |



Also consider

```
?p wdt:P31/wdt:P279* wd:Q131569  . 
```

is the BGP to retrieve all **treaties**

## Workload Goals


1. Identify the BGP to retrieve treaties that took place in E.U. countries

2. Identify the BGP for obtaining which countries or people signed the treaties

3. How many treaties have been published per year?

4. For how many treaties we know causes or effects?

5. Analyze the number of countries and how many treaties they have signed and their causes
 
   5.1 Are there two countries that signed many common treaties?
   
   5.2 Which are the treaties with most signatories? Which country signed the most treaties?
   
   5.3 Are there countries that did not sign a treaty with the U.S. but signed a treaty with Germany?


In [ ]:
# start your workflow here

In [5]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P31/wdt:P279* wd:Q131569  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '3962')]


1

In [6]:
#return treaties

queryString = """
SELECT ?treaties ?treatiesName
WHERE { 

    ?treaties wdt:P31/wdt:P279* wd:Q131569  . 

    ?treaties <http://schema.org/name> ?treatiesName
} 
"""

print("Results")
run_query(queryString)

Results
[('treaties', 'http://www.wikidata.org/entity/Q16210164'), ('treatiesName', 'Treaty of Fort Adams')]
[('treaties', 'http://www.wikidata.org/entity/Q17013654'), ('treatiesName', 'Penal Sanctions')]
[('treaties', 'http://www.wikidata.org/entity/Q17035581'), ('treatiesName', 'Treaty of Washington City')]
[('treaties', 'http://www.wikidata.org/entity/Q910554'), ('treatiesName', 'Dublin Regulation')]
[('treaties', 'http://www.wikidata.org/entity/Q189066'), ('treatiesName', 'Treaty establishing a Constitution for Europe')]
[('treaties', 'http://www.wikidata.org/entity/Q668645'), ('treatiesName', 'Surrender of Caserta')]
[('treaties', 'http://www.wikidata.org/entity/Q18647563'), ('treatiesName', 'Treaty on European Union')]
[('treaties', 'http://www.wikidata.org/entity/Q22908408'), ('treatiesName', 'Russian–Serbian Alliance of 1807')]
[('treaties', 'http://www.wikidata.org/entity/Q26833034'), ('treatiesName', 'New Zealand–France Marine Delimitation Agreement')]
[('treaties', 'http://w

2926

In [3]:
#return predicates connected to treaties

queryString = """
SELECT DISTINCT ?predName ?pred
WHERE { 

    ?treaties wdt:P31/wdt:P279* wd:Q131569;
        ?pred ?obj.

    ?pred <http://schema.org/name> ?predName.
    ?obj <http://schema.org/name> ?objName.
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'Wikidata property'), ('pred', 'http://www.wikidata.org/prop/direct/P1687')]
[('predName', 'addressee'), ('pred', 'http://www.wikidata.org/prop/direct/P1817')]
[('predName', 'amended by'), ('pred', 'http://www.wikidata.org/prop/direct/P2567')]
[('predName', 'applies to jurisdiction'), ('pred', 'http://www.wikidata.org/prop/direct/P1001')]
[('predName', 'applies to part'), ('pred', 'http://www.wikidata.org/prop/direct/P518')]
[('predName', 'archives at'), ('pred', 'http://www.wikidata.org/prop/direct/P485')]
[('predName', 'author'), ('pred', 'http://www.wikidata.org/prop/direct/P50')]
[('predName', 'authority'), ('pred', 'http://www.wikidata.org/prop/direct/P797')]
[('predName', 'based on'), ('pred', 'http://www.wikidata.org/prop/direct/P144')]
[('predName', 'basic form of government'), ('pred', 'http://www.wikidata.org/prop/direct/P122')]
[('predName', 'capital'), ('pred', 'http://www.wikidata.org/prop/direct/P36')]
[('predName', 'catalog'), ('pred', 'http://www.w

109

Info for Query 3
* [('predName', 'publication date'), ('pred', 'http://www.wikidata.org/prop/direct/P577')], ('obj', '0855-09-23T00:00:00Z')]

In [13]:
#return predicates connected to treaties (outcoming edges)

queryString = """
SELECT DISTINCT ?predName ?pred
WHERE { 

    ?treaties wdt:P31/wdt:P279* wd:Q131569.
    ?sub ?pred ?treaties.

    ?pred <http://schema.org/name> ?predName.
    ?sub <http://schema.org/name> ?subName.
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'Wikidata item of this property'), ('pred', 'http://www.wikidata.org/prop/direct/P1629')]
[('predName', 'Wikidata property example'), ('pred', 'http://www.wikidata.org/prop/direct/P1855')]
[('predName', 'addressee'), ('pred', 'http://www.wikidata.org/prop/direct/P1817')]
[('predName', 'advertises'), ('pred', 'http://www.wikidata.org/prop/direct/P6291')]
[('predName', 'affiliation'), ('pred', 'http://www.wikidata.org/prop/direct/P1416')]
[('predName', 'amended by'), ('pred', 'http://www.wikidata.org/prop/direct/P2567')]
[('predName', "applicable 'stated in' value"), ('pred', 'http://www.wikidata.org/prop/direct/P9073')]
[('predName', 'applies to jurisdiction'), ('pred', 'http://www.wikidata.org/prop/direct/P1001')]
[('predName', 'authority'), ('pred', 'http://www.wikidata.org/prop/direct/P797')]
[('predName', 'based on'), ('pred', 'http://www.wikidata.org/prop/direct/P144')]
[('predName', 'category combines topics'), ('pred', 'http://www.wikidata.org/prop/direct/P9

82

In [11]:
#return predicates and subjects connected to treaties

queryString = """
SELECT DISTINCT ?predName ?pred ?sub
WHERE { 

    ?treaties wdt:P31/wdt:P279* wd:Q131569.
    ?sub ?pred ?treaties.

    ?pred <http://schema.org/name> ?predName.
} 
ORDER BY ?sub
"""

print("Results")
run_query(queryString)

Results
[('predName', 'Wikidata property example'), ('pred', 'http://www.wikidata.org/prop/direct/P1855'), ('sub', 'http://www.wikidata.org/entity/P1001')]
[('predName', 'Wikidata property example'), ('pred', 'http://www.wikidata.org/prop/direct/P1855'), ('sub', 'http://www.wikidata.org/entity/P1891')]
[('predName', 'Wikidata property example'), ('pred', 'http://www.wikidata.org/prop/direct/P1855'), ('sub', 'http://www.wikidata.org/entity/P2058')]
[('predName', 'Wikidata property example'), ('pred', 'http://www.wikidata.org/prop/direct/P1855'), ('sub', 'http://www.wikidata.org/entity/P2567')]
[('predName', 'Wikidata property example'), ('pred', 'http://www.wikidata.org/prop/direct/P1855'), ('sub', 'http://www.wikidata.org/entity/P3889')]
[('predName', 'Wikidata property example'), ('pred', 'http://www.wikidata.org/prop/direct/P1855'), ('sub', 'http://www.wikidata.org/entity/P3968')]
[('predName', 'Wikidata property example'), ('pred', 'http://www.wikidata.org/prop/direct/P1855'), ('sub

2085

Info:
* [('objName', 'Europe'), ('predName', 'applies to jurisdiction'), ('obj', 'http://www.wikidata.org/entity/Q46'), ('pred', 'http://www.wikidata.org/prop/direct/P1001')]
* [('objName', 'Italy'), ('predName', 'applies to jurisdiction'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('pred', 'http://www.wikidata.org/prop/direct/P1001')]
<br><br>
Query 2
* [('objName', 'Italy'), ('predName', 'signatory'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('pred', 'http://www.wikidata.org/prop/direct/P1891')]
<br><br>
Query 4
* [('predName', 'has cause'), ('pred', 'http://www.wikidata.org/prop/direct/P828')]
* [('predName', 'has effect'), ('pred', 'http://www.wikidata.org/prop/direct/P1542')]
* [('predName', 'has immediate cause'), ('pred', 'http://www.wikidata.org/prop/direct/P1478')]

In [20]:
#return predicates and objects connected to Italy of treaties
#?objName ?obj
queryString = """
SELECT DISTINCT ?predName ?pred 
WHERE { 

    ?treaties wdt:P31/wdt:P279* wd:Q131569;
        wdt:P1001 wd:Q38.
    wd:Q38 ?pred ?obj.

    ?pred <http://schema.org/name> ?predName.
    ?obj <http://schema.org/name> ?objName.
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'Wikimedia outline'), ('pred', 'http://www.wikidata.org/prop/direct/P5125')]
[('predName', 'anthem'), ('pred', 'http://www.wikidata.org/prop/direct/P85')]
[('predName', 'basic form of government'), ('pred', 'http://www.wikidata.org/prop/direct/P122')]
[('predName', 'capital'), ('pred', 'http://www.wikidata.org/prop/direct/P36')]
[('predName', 'category for films shot at this location'), ('pred', 'http://www.wikidata.org/prop/direct/P1740')]
[('predName', 'category for maps'), ('pred', 'http://www.wikidata.org/prop/direct/P7867')]
[('predName', 'category for people who died here'), ('pred', 'http://www.wikidata.org/prop/direct/P1465')]
[('predName', 'category for people born here'), ('pred', 'http://www.wikidata.org/prop/direct/P1464')]
[('predName', 'category of associated people'), ('pred', 'http://www.wikidata.org/prop/direct/P1792')]
[('predName', 'category of people buried here'), ('pred', 'http://www.wikidata.org/prop/direct/P1791')]
[('predName', 'coat of ar

58

In [ ]:
#return the treaties in european countries

queryString = """
SELECT DISTINCT ?treatiesName
WHERE { 

    ?treaties wdt:P31/wdt:P279* wd:Q131569;
        wdt:P17 ?countries.
    ?countries wdt:P463 wd:Q458.

    ?treaties <http://schema.org/name> ?treatiesName.
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

'member of' (wdt:P463')  'European Union'  wd:Q458'

**********

# Query 1
Identify the BGP to retrieve treaties that took place in E.U. countries.

In [9]:
#return the treaties in european countries

queryString = """
SELECT DISTINCT ?treatiesName ?countriesName
WHERE { 

    ?treaties wdt:P31/wdt:P279* wd:Q131569;
        wdt:P17 ?countries.
    ?countries wdt:P463 wd:Q458.

    ?treaties <http://schema.org/name> ?treatiesName.
    ?countries <http://schema.org/name> ?countriesName.
} 
ORDER BY ?treatiesName
"""

print("Results")
run_query(queryString)

Results
[('treatiesName', '1996 France–United Kingdom Maritime Delimitation Agreements'), ('countriesName', 'France')]
[('treatiesName', '2006 Franco–Italian–Spanish Middle East Peace Plan'), ('countriesName', 'Spain')]
[('treatiesName', 'Agreement between the Allied and Associated Powers with Regard to the Contribution to the Cost of Liberation of the Territories of the Former Austro-Hungarian Monarchy'), ('countriesName', 'France')]
[('treatiesName', 'Anglo-Austrian Treaty of 1800'), ('countriesName', 'Austria')]
[('treatiesName', 'Anglo-Irish Agreement'), ('countriesName', 'United Kingdom')]
[('treatiesName', 'Anglo-Irish Agreement'), ('countriesName', 'Ireland')]
[('treatiesName', 'Anglo-Portuguese Treaty of 1878'), ('countriesName', 'Portugal')]
[('treatiesName', 'Anglo-Siamese extradition treaty of 1911'), ('countriesName', 'United Kingdom')]
[('treatiesName', 'Article 231 of the Treaty of Versailles'), ('countriesName', 'France')]
[('treatiesName', 'Balkan Pact'), ('countriesNam

251

> Initially, I've searched the predicates and objects connected to treaties. Then I've searched the subjects and predicates connected to them.
>
> Then I've searched something in order to interpret 'took place in EU'. So, I've found the wdt:P17 *country* and then, I've searched a way to take only the countries in the EU (wdt:P453 *member of*) and wd:Q458 *European Union*.

> In the query, I've returned the treaties and also the countries.
**********

In [15]:
#return the treaties and their signatory

queryString = """
SELECT DISTINCT ?treatiesName ?countriesSignName 
WHERE { 

    ?treaties wdt:P31/wdt:P279* wd:Q131569;
        wdt:P1891 ?countriesSign.

    ?treaties <http://schema.org/name> ?treatiesName.
    ?countriesSign <http://schema.org/name> ?countriesSignName.
} 
ORDER BY ?countriesSignName
"""

print("Results")
run_query(queryString)

Results
[('treatiesName', 'Regional Comprehensive Economic Partnership'), ('countriesSignName', 'ASEAN')]
[('treatiesName', 'Southeast Asian Nuclear-Weapon-Free Zone Treaty'), ('countriesSignName', 'ASEAN')]
[('treatiesName', 'Treaty of Fes'), ('countriesSignName', 'Abd al-Hafid of Morocco')]
[('treatiesName', 'Bahrain–Israel peace agreement'), ('countriesSignName', 'Abdullatif bin Rashid Al Zayani')]
[('treatiesName', 'Munich agreement'), ('countriesSignName', 'Adolf Hitler')]
[('treatiesName', 'Treaty of Frankfurt'), ('countriesSignName', 'Adolphe Thiers')]
[('treatiesName', 'International Air Services Transit Agreement'), ('countriesSignName', 'Afghanistan')]
[('treatiesName', 'Energy Charter Treaty'), ('countriesSignName', 'Afghanistan')]
[('treatiesName', 'Marrakesh Treaty to Facilitate Access to Published Works for Persons Who Are Blind, Visually Impaired or Otherwise Print Disabled'), ('countriesSignName', 'Afghanistan')]
[('treatiesName', 'Hague Protocol'), ('countriesSignName'

3050

In [43]:
#return the objects connected to instance of 

queryString = """
SELECT DISTINCT ?obj ?objName  
WHERE { 

    ?treaties wdt:P31/wdt:P279* wd:Q131569;
        wdt:P1891 ?countriesSign.
    ?countriesSign wdt:P31 ?obj.

    ?obj <http://schema.org/name> ?objName.
    ?countriesSign <http://schema.org/name> ?countriesSignName.
} 
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q93342462'), ('objName', 'Ancient Greek archaeological site')]
[('obj', 'http://www.wikidata.org/entity/Q1147274'), ('objName', 'Apostolic see')]
[('obj', 'http://www.wikidata.org/entity/Q76034'), ('objName', 'Canadian Shield')]
[('obj', 'http://www.wikidata.org/entity/Q12857432'), ('objName', 'Chinese dynasty')]
[('obj', 'http://www.wikidata.org/entity/Q202686'), ('objName', 'Commonwealth realm')]
[('obj', 'http://www.wikidata.org/entity/Q16850120'), ('objName', 'Council of Ministers')]
[('obj', 'http://www.wikidata.org/entity/Q26236686'), ('objName', 'East India Company')]
[('obj', 'http://www.wikidata.org/entity/Q27925057'), ('objName', 'Electoral Principate')]
[('obj', 'http://www.wikidata.org/entity/Q1450662'), ('objName', 'Government of France')]
[('obj', 'http://www.wikidata.org/entity/Q672729'), ('objName', 'Islamic republic')]
[('obj', 'http://www.wikidata.org/entity/Q51576574'), ('objName', 'Mediterranean country')]
[('obj', 'h

162

In [45]:
#return the objects connected to instance of with italy

queryString = """
SELECT DISTINCT ?obj ?objName  
WHERE { 

    ?treaties wdt:P31/wdt:P279* wd:Q131569;
        wdt:P1891  wd:Q38.
     wd:Q38 wdt:P31 ?obj.

    ?obj <http://schema.org/name> ?objName.
} 
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q6256'), ('objName', 'country')]
[('obj', 'http://www.wikidata.org/entity/Q3624078'), ('objName', 'sovereign state')]


2

*********
# Query 2
Identify the BGP for obtaining which countries or people signed the treaties

In [15]:
#return the treaties and country/people that signed them (put in a list)

queryString = """
SELECT DISTINCT ?treatiesName (GROUP_CONCAT(DISTINCT ?signName ; separator=",") AS ?listOfCountries)
WHERE { 

    ?treaties wdt:P31/wdt:P279* wd:Q131569;
        wdt:P1891 ?sign.
        
    ?treaties <http://schema.org/name> ?treatiesName.
    ?sign <http://schema.org/name> ?signName.
} 
GROUP BY ?treatiesName
ORDER BY ?treatiesName

"""

print("Results")
run_query(queryString)

Results
[('treatiesName', '1828 Treaty of Montevideo'), ('listOfCountries', 'Empire of Brazil,United Provinces of the Río de la Plata')]
[('treatiesName', '1893 border protocol between Argentina & Chile'), ('listOfCountries', 'Isidoro Errázuriz,Norberto Quirno Costa')]
[('treatiesName', '1950 Indo-Nepal Treaty of Peace and Friendship'), ('listOfCountries', 'India,Kingdom of Nepal')]
[('treatiesName', '1951 Polish–Soviet territorial exchange'), ('listOfCountries', "Polish People's Republic,Soviet Union")]
[('treatiesName', '1971 Okinawa Reversion Agreement'), ('listOfCountries', 'Kiichi Aichi,William P. Rogers')]
[('treatiesName', '1975 Algiers Agreement'), ('listOfCountries', 'Mohammad Reza Pahlavi,Saddam Hussein')]
[('treatiesName', '1991 Paris Peace Accords'), ('listOfCountries', "Australia,Brunei,Buddhist Liberal Democratic Party,Cambodian People's Party,Canada,Communist Party of Kampuchea,France,Funcinpec Party,India,Indonesia,Japan,Laos,Malaysia,Non-Aligned Movement,People's Repub

679

> In this case, I've search in the first queries and I've found wdt:P1891 *signatory*.

************

In [19]:
#return the publication year and the treaties

queryString = """
SELECT DISTINCT ?treatiesName (year(xsd:dateTime(?publicationYear)) AS ?pubYear)
WHERE { 

    ?treaties wdt:P31/wdt:P279* wd:Q131569;
        wdt:P577 ?publicationYear.
        
    ?treaties <http://schema.org/name> ?treatiesName.
} 
ORDER BY ?obj

"""

print("Results")
run_query(queryString)

Results
[('treatiesName', 'International Covenant on Economic, Social and Cultural Rights'), ('pubYear', '1966')]
[('treatiesName', 'Convention on the Rights of the Child'), ('pubYear', '1989')]
[('treatiesName', 'Treaty on the Prohibition of Nuclear Weapons'), ('pubYear', '2017')]
[('treatiesName', 'Secret Additional Protocol to the Molotov–Ribbentrop Pact'), ('pubYear', '1939')]
[('treatiesName', 'Second Geneva Convention'), ('pubYear', '1949')]
[('treatiesName', 'Agreement between the Parties to the North Atlantic Treaty regarding the Status of their Forces'), ('pubYear', '1951')]
[('treatiesName', 'Minamata Convention on Mercury'), ('pubYear', '2013')]
[('treatiesName', 'European Agreement concerning the International Carriage of Dangerous Goods by Road'), ('pubYear', '1957')]
[('treatiesName', 'Munich agreement'), ('pubYear', '1939')]
[('treatiesName', 'United Nations Framework Convention on Climate Change'), ('pubYear', '1992')]
[('treatiesName', 'International Covenant on Civil 

94

***********
# Query 3
How many treaties have been published per year?

In [57]:
#count the treaties per year
queryString = """
SELECT (COUNT(?treatiesName) AS ?howmanyTreaties) (year(xsd:dateTime(?publicationYear)) AS ?pubYear)
WHERE { 

    ?treaties wdt:P31/wdt:P279* wd:Q131569;
        wdt:P577 ?publicationYear.
        
    ?treaties <http://schema.org/name> ?treatiesName.
} 
GROUP BY ?publicationYear
ORDER BY ?publicationYear
"""

print("Results")
run_query(queryString)

Results
[('howmanyTreaties', '1'), ('pubYear', '843')]
[('howmanyTreaties', '1'), ('pubYear', '855')]
[('howmanyTreaties', '1'), ('pubYear', '1817')]
[('howmanyTreaties', '2'), ('pubYear', '1842')]
[('howmanyTreaties', '1'), ('pubYear', '1859')]
[('howmanyTreaties', '1'), ('pubYear', '1864')]
[('howmanyTreaties', '1'), ('pubYear', '1871')]
[('howmanyTreaties', '1'), ('pubYear', '1871')]
[('howmanyTreaties', '1'), ('pubYear', '1881')]
[('howmanyTreaties', '1'), ('pubYear', '1884')]
[('howmanyTreaties', '1'), ('pubYear', '1886')]
[('howmanyTreaties', '1'), ('pubYear', '1894')]
[('howmanyTreaties', '1'), ('pubYear', '1898')]
[('howmanyTreaties', '1'), ('pubYear', '1900')]
[('howmanyTreaties', '1'), ('pubYear', '1904')]
[('howmanyTreaties', '1'), ('pubYear', '1911')]
[('howmanyTreaties', '2'), ('pubYear', '1918')]
[('howmanyTreaties', '1'), ('pubYear', '1918')]
[('howmanyTreaties', '1'), ('pubYear', '1921')]
[('howmanyTreaties', '1'), ('pubYear', '1926')]
[('howmanyTreaties', '1'), ('pubYe

89

> I've searched in the initial queries and I've found wdt:P577 *publication year*

**********

In [82]:
#return cause, effects, immediate cause of treaties
queryString = """
SELECT ?treatiesName ?causeName ?effectName ?immCauseName
WHERE { 
    ?treaties wdt:P31/wdt:P279* wd:Q131569.
    ?treaties wdt:P828 ?cause;
        wdt:P1542 ?effect;
        wdt:P1478 ?immCause.
        
    ?treaties <http://schema.org/name> ?treatiesName.
    ?cause <http://schema.org/name> ?causeName.
    ?effect <http://schema.org/name> ?effectName.
    ?immCause <http://schema.org/name> ?immCauseName.
} 
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [71]:
#return immediate cause of treaties
queryString = """
SELECT ?treatiesName ?causeName ?effectName ?immCauseName
WHERE { 
    ?treaties wdt:P31/wdt:P279* wd:Q131569.
    ?treaties wdt:P1478 ?immCause.
        
    ?treaties <http://schema.org/name> ?treatiesName.
    ?immCause <http://schema.org/name> ?immCauseName.
} 
"""

print("Results")
run_query(queryString)

Results
[('treatiesName', 'Bretton Woods system'), ('immCauseName', 'Bretton Woods Conference')]
[('treatiesName', 'Treaty of Nanjing'), ('immCauseName', 'First Opium War')]
[('treatiesName', 'Treaty of Nanjing'), ('immCauseName', 'First Opium War')]
[('treatiesName', 'Treaty of Guadalupe Hidalgo'), ('immCauseName', 'territorial expansion')]


4

In [76]:
#return cause of treaties
queryString = """
SELECT ?treatiesName ?causeName 
WHERE { 
    ?treaties wdt:P31/wdt:P279* wd:Q131569.
    ?treaties wdt:P828 ?Cause.
        
    ?treaties <http://schema.org/name> ?treatiesName.
    ?Cause <http://schema.org/name> ?CauseName.
} 
"""

print("Results")
run_query(queryString)

Results
[('treatiesName', 'Munich agreement')]
[('treatiesName', 'Munich agreement')]
[('treatiesName', 'London Declaration')]
[('treatiesName', 'Charter of the United Nations')]
[('treatiesName', 'Treaty of Fontainebleau')]
[('treatiesName', 'Paris Agreement')]
[('treatiesName', 'Cusco Declaration')]
[('treatiesName', 'BRICS Contingent Reserve Arrangement')]
[('treatiesName', 'Act of Algeciras')]
[('treatiesName', 'Traité de Saint-Julien')]
[('treatiesName', 'Partitio terrarum imperii Romaniae')]
[('treatiesName', 'Hague Conventions of 1899 and 1907')]
[('treatiesName', 'Treaty of Paris')]
[('treatiesName', 'Treaty of Paris')]
[('treatiesName', 'Treaty of Paris')]
[('treatiesName', 'Treaty of Paris')]
[('treatiesName', 'Treaty of Jassy')]
[('treatiesName', 'Treaty of Paris')]
[('treatiesName', 'Peace of Paris (1783)')]
[('treatiesName', 'Egypt–Israel Peace Treaty')]
[('treatiesName', 'Treaty of Versailles')]
[('treatiesName', 'Treaty of Paris')]
[('treatiesName', 'Brioni Agreement')]


37

In [80]:
#return the treaties checking if cause, effects, immediate cause of treaties exist
queryString = """
SELECT ?treatiesName
WHERE { 
    ?treaties wdt:P31/wdt:P279* wd:Q131569.
    
    FILTER EXISTS{?treaties wdt:P828|wdt:P1542|wdt:P1478 ?causeEffect}.
    ?treaties <http://schema.org/name> ?treatiesName.
} 
"""

print("Results")
run_query(queryString)

Results
[('treatiesName', 'Munich agreement')]
[('treatiesName', 'Rome Statute of the International Criminal Court')]
[('treatiesName', 'Évian Accords')]
[('treatiesName', 'London Declaration')]
[('treatiesName', 'Charter of the United Nations')]
[('treatiesName', 'Treaty of Fontainebleau')]
[('treatiesName', 'Vertrag von Nymphenburg')]
[('treatiesName', 'Paris Agreement')]
[('treatiesName', 'Cusco Declaration')]
[('treatiesName', 'BRICS Contingent Reserve Arrangement')]
[('treatiesName', 'Act of Algeciras')]
[('treatiesName', 'Bretton Woods system')]
[('treatiesName', 'Covenant for the formation of Rajasthan')]
[('treatiesName', 'Chapter VII of the United Nations Charter')]
[('treatiesName', 'Pan-American Treaty')]
[('treatiesName', 'Warsaw Treaty')]
[('treatiesName', 'Convention on the International Maritime Organization')]
[('treatiesName', 'Boundary Treaty of 1866 between Chile and Bolivia')]
[('treatiesName', 'Traité de Saint-Julien')]
[('treatiesName', 'Partitio terrarum imperii 

56

***********

# Query 4
For how many treaties we know causes or effects?

In [81]:
#return the count of treaties checking if cause, effects, immediate cause of treaties exist
queryString = """
SELECT COUNT(?treatiesName)
WHERE { 
    ?treaties wdt:P31/wdt:P279* wd:Q131569.
    
    FILTER EXISTS{?treaties wdt:P828|wdt:P1542|wdt:P1478 ?causeEffect}.
    ?treaties <http://schema.org/name> ?treatiesName.
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '56')]


1

> In the initial queries, I've found:
> *wdt:P828 *has cause*
> * wdt:P1542 *has effect*
> *wdt:P1478 *has immediate cause*
>
> Initially, I've put everything in a single query, but the result is empty. So, I've run a single query for each predicate.
> I've decide to count the number of treaties, that has one of these predicates, so I've inserted a FILTER EXISTS in which the treaties are connected through wdt:P828 or wdt:P1542 or wdt:P1478. 

*********

In [2]:
#return the treaties with the count of signature
queryString = """
SELECT ?treatiesName (COUNT(?countriesSign) AS ?howmanySign)
WHERE { 
    ?treaties wdt:P31/wdt:P279* wd:Q131569;
        wdt:P1891 ?countriesSign.

    ?treaties <http://schema.org/name> ?treatiesName.
} 
GROUP BY ?treatiesName
"""

print("Results")
run_query(queryString)

Results
[('treatiesName', 'Berne Convention for the Protection of Literary and Artistic Works'), ('howmanySign', '9')]
[('treatiesName', '1951 Polish–Soviet territorial exchange'), ('howmanySign', '2')]
[('treatiesName', 'Treaty of Alcañices'), ('howmanySign', '2')]
[('treatiesName', 'Convention on the Elimination of All Forms of Discrimination against Women'), ('howmanySign', '2')]
[('treatiesName', 'Union of Horodło'), ('howmanySign', '2')]
[('treatiesName', 'Nice Agreement Concerning the International Classification of Goods and Services for the Purposes of the Registration of Marks'), ('howmanySign', '22')]
[('treatiesName', 'Treaty of Georgievsk'), ('howmanySign', '2')]
[('treatiesName', 'Treaty of friendship and alliance between the Government of Mongolia and Tibet'), ('howmanySign', '2')]
[('treatiesName', 'Singapore Treaty on the Law of Trademarks'), ('howmanySign', '53')]
[('treatiesName', 'Japan-Indonesia Economic Partnership Agreement'), ('howmanySign', '6')]
[('treatiesName

680

**********
# Query 5.1 
Are there two countries that signed many common treaties?

In [39]:
#return the treaties in european countries

queryString = """ 
ASK WHERE {
    SELECT ?howmany 
    WHERE {
            SELECT DISTINCT ?countries1Name ?countries2Name (COUNT(?treatiesName) AS ?howmany)
            WHERE { 

                ?treaties wdt:P31/wdt:P279* wd:Q131569;
                    wdt:P17 ?countries1;
                    wdt:P17 ?countries2.

                FILTER(?countries1 > ?countries2).
                ?treaties <http://schema.org/name> ?treatiesName.
                ?countries1 <http://schema.org/name> ?countries1Name.
                ?countries2 <http://schema.org/name> ?countries2Name.
            } 
            GROUP BY ?countries1Name ?countries2Name
        }
    GROUP BY ?howmany
    HAVING (?howmany > 5)       
    ORDER BY DESC (?howmany)
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

> I've taken the BGP from the first query.

*******
# Query 5.2
## Part 1
Which are the treaties with most signatories?

In [5]:
#return the five treaties with most signature
queryString = """
SELECT ?treatiesName (COUNT(?countriesSign) AS ?howmanySign)
WHERE { 
    ?treaties wdt:P31/wdt:P279* wd:Q131569;
        wdt:P1891 ?countriesSign.

    ?treaties <http://schema.org/name> ?treatiesName.
} 
GROUP BY ?treatiesName
ORDER BY DESC (?howmanySign)
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('treatiesName', 'Universal Copyright Convention'), ('howmanySign', '97')]
[('treatiesName', 'Treaty on the Prohibition of Nuclear Weapons'), ('howmanySign', '86')]
[('treatiesName', 'Marrakesh Treaty to Facilitate Access to Published Works for Persons Who Are Blind, Visually Impaired or Otherwise Print Disabled'), ('howmanySign', '80')]
[('treatiesName', 'Beijing Treaty on Audiovisual Performances'), ('howmanySign', '74')]
[('treatiesName', 'Patent Law Treaty'), ('howmanySign', '54')]


5

> I've taken the above query and modify to return the number of signature.


## part 2
Which country signed the most treaties?

In [47]:
#return the country with most treaties
queryString = """
SELECT ?countriesSignName (COUNT(?treaties) AS ?numberTreaties)
WHERE { 
    ?treaties wdt:P31/wdt:P279* wd:Q131569;
        wdt:P1891 ?countriesSign.
    ?countriesSign wdt:P31 wd:Q6256.

    ?countriesSign <http://schema.org/name> ?countriesSignName.
} 
GROUP BY ?countriesSignName
ORDER BY DESC (?numberTreaties)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('countriesSignName', 'France'), ('numberTreaties', '214')]


1

> I've added wdt:P31 *instance of*, because in the objects connected through wdt:P1891 are countries and also people.
*********

In [23]:
#return the treaties where there is not US
queryString = """
SELECT DISTINCT ?treaties
WHERE { 
    ?treaties wdt:P31/wdt:P279* wd:Q131569;
             wdt:P1891 ?countryNoUS.
    
    FILTER NOT EXISTS{?treaties wdt:P1891 wd:Q30}.

    ?countryNoUS <http://schema.org/name> ?countryNoUSName.
} 

"""

print("Results")
run_query(queryString)

Results
[('treaties', 'http://www.wikidata.org/entity/Q87405230')]
[('treaties', 'http://www.wikidata.org/entity/Q931982')]
[('treaties', 'http://www.wikidata.org/entity/Q99207513')]
[('treaties', 'http://www.wikidata.org/entity/Q87405170')]
[('treaties', 'http://www.wikidata.org/entity/Q87405240')]
[('treaties', 'http://www.wikidata.org/entity/Q154255')]
[('treaties', 'http://www.wikidata.org/entity/Q19224010')]
[('treaties', 'http://www.wikidata.org/entity/Q158021')]
[('treaties', 'http://www.wikidata.org/entity/Q1568540')]
[('treaties', 'http://www.wikidata.org/entity/Q7843235')]
[('treaties', 'http://www.wikidata.org/entity/Q87405191')]
[('treaties', 'http://www.wikidata.org/entity/Q30888316')]
[('treaties', 'http://www.wikidata.org/entity/Q87405208')]
[('treaties', 'http://www.wikidata.org/entity/Q47142474')]
[('treaties', 'http://www.wikidata.org/entity/Q3313588')]
[('treaties', 'http://www.wikidata.org/entity/Q87405134')]
[('treaties', 'http://www.wikidata.org/entity/Q48954495')

889

In [26]:
#return the treaties where there is Germany
queryString = """
SELECT DISTINCT ?treaties
WHERE { 
    ?treaties wdt:P31/wdt:P279* wd:Q131569;
             wdt:P1891 ?countryYesGerm.
    
    FILTER EXISTS{?treaties wdt:P1891 wd:Q183}.

    ?countryYesGerm <http://schema.org/name> ?countryYesGermName.
} 

"""

print("Results")
run_query(queryString)

Results
[('treaties', 'http://www.wikidata.org/entity/Q844919')]
[('treaties', 'http://www.wikidata.org/entity/Q30101455')]
[('treaties', 'http://www.wikidata.org/entity/Q100825757')]
[('treaties', 'http://www.wikidata.org/entity/Q100825794')]
[('treaties', 'http://www.wikidata.org/entity/Q11418793')]
[('treaties', 'http://www.wikidata.org/entity/Q369989')]
[('treaties', 'http://www.wikidata.org/entity/Q15991255')]
[('treaties', 'http://www.wikidata.org/entity/Q77724480')]
[('treaties', 'http://www.wikidata.org/entity/Q4881201')]
[('treaties', 'http://www.wikidata.org/entity/Q5166228')]
[('treaties', 'http://www.wikidata.org/entity/Q1056771')]
[('treaties', 'http://www.wikidata.org/entity/Q98487293')]
[('treaties', 'http://www.wikidata.org/entity/Q98489540')]
[('treaties', 'http://www.wikidata.org/entity/Q98489466')]
[('treaties', 'http://www.wikidata.org/entity/Q2511409')]
[('treaties', 'http://www.wikidata.org/entity/Q741547')]
[('treaties', 'http://www.wikidata.org/entity/Q47359')]


77

In [28]:
#return the treaties where there is Germany, but not US
queryString = """
SELECT DISTINCT ?countryYesGermNoUSName
WHERE { 
    ?treaties wdt:P31/wdt:P279* wd:Q131569;
             wdt:P1891 ?countryYesGermNoUS.
    
    FILTER EXISTS{?treaties wdt:P1891 wd:Q183}.
    FILTER NOT EXISTS{?treaties wdt:P1891 wd:Q30}.

    ?countryYesGermNoUS <http://schema.org/name> ?countryYesGermNoUSName.
} 

"""

print("Results")
run_query(queryString)

Results
[('countryYesGermNoUSName', 'Danish Realm')]
[('countryYesGermNoUSName', 'World Communion of Reformed Churches')]
[('countryYesGermNoUSName', 'Morocco')]
[('countryYesGermNoUSName', 'Niger')]
[('countryYesGermNoUSName', 'Senegal')]
[('countryYesGermNoUSName', 'Seychelles')]
[('countryYesGermNoUSName', 'Kenya')]
[('countryYesGermNoUSName', 'France')]
[('countryYesGermNoUSName', 'United Kingdom')]
[('countryYesGermNoUSName', "People's Republic of China")]
[('countryYesGermNoUSName', 'Brazil')]
[('countryYesGermNoUSName', 'Russia')]


12

***********
# Query 5.3
Are there countries that did not sign a treaty with the U.S. but signed a treaty with Germany?
**NOTE**: the actual German.

In [29]:
#return the treaties where there is Germany, but not US
queryString = """
ASK WHERE{
            SELECT DISTINCT ?countryYesGermNoUSName
            WHERE { 
                ?treaties wdt:P31/wdt:P279* wd:Q131569;
                         wdt:P1891 ?countryYesGermNoUS.

                FILTER EXISTS{?treaties wdt:P1891 wd:Q183}.
                FILTER NOT EXISTS{?treaties wdt:P1891 wd:Q30}.

                ?countryYesGermNoUS <http://schema.org/name> ?countryYesGermNoUSName.
            } 
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

> Used the information in the initial queries.
**********